In [1]:
from dotenv import load_dotenv
import os
# Load environment variables from .env file
load_dotenv()

True

In [2]:
from langchain_community.document_loaders import PyPDFLoader
import pprint


file_path="./data/machine_learning.pdf"
loader = PyPDFLoader(file_path)
docs = loader.load()
docs[0]
pprint.pp(docs[0].metadata)

{'producer': 'PyPDF',
 'creator': 'PyPDF',
 'creationdate': '',
 'source': './data/machine_learning.pdf',
 'total_pages': 120,
 'page': 0,
 'page_label': '1'}


In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # chunk size (characters)
    chunk_overlap=200,  # chunk overlap (characters)
    add_start_index=True,  # track index in original document
)
all_splits = text_splitter.split_documents(docs)

print(f"Split blog post into {len(all_splits)} sub-documents.")

In [ ]:
# Import the Pinecone library
from pinecone import Pinecone

# Initialize a Pinecone client with your API key
PINECONE_API=os.getenv("PINECONE_API_KEY")
pc = Pinecone(api_key=PINECONE_API)

In [ ]:
from pinecone import ServerlessSpec

index_name=os.getenv("PINECONE_INDEX_NAME")
region_name=os.getenv("PINECONE_ENVIRONMENT")
if not pc.has_index(index_name):
    pc.create_index(
        name=index_name,
        dimension=768,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region=region_name),
    )

index = pc.Index(index_name)

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
google_api_key = os.getenv("GOOGLE_API_KEY")
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=google_api_key)

In [ ]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=embeddings)